## Scenario/Stakeholder Based Analysis of NYC taxi rides data
##### Authors: Panini Mokrala, Dmitrii Danilov

# Introduction

Through this project, we will investigate the decisions that various stakeholders in a Taxi Ecosystem take and overlay them with the varying weather conditions within New York City to check for correlation. We will take a closer look at the following definitions/assumptions before getting into questions-

1.   Stakeholders - We will consider the Commuters travelling in the Taxis and Taxi owners/drivers as stakeholders
2.   Decisions - Depending on the stakeholders we chose, there can be different decisions - for example, Taxi drivers can prefer fewer trips in a winter season compared to a summer season while commuters would prefer a taxi ride more often in a winter season compared to a summer season
3. Time period/ other assumptions  - We will be utilizing the taxi rides information between 2017 to 2019 to highlight the trends. There can be few interactive effects with the entry of Uber/Lyft. Due to data reliability issues, we will not be able to attribute this interactive effect

Now that we have a clearer understanding of the problem space, there are two main areas that we would like to address through this project depending on the stakeholder -

1. If you are a traveler or commuter - You can plan your trip by answering:
    a. How does the average fare/trip vary across the various taxi zones of New York with changing timeline and weather conditions?
    b. What is the average time/trip between two points in New York? How does that change with the weather variations across the time?
    c. At a given point in time and at a given temperature range, how does the availability of the taxis vary? (Please note: Availability is defined as number of active rides at a given point in time)
2. If you want to help a taxi driver/owner - You can help them plan their next season by answering : 
    a. How do the taxi availability vary across the various taxi zones? How do the weather conditions relate with the taxi availability over time? 
    b. What can be a reasonable fare estimate / trip that the owner can quote between the origin and destination?

While we enlisted the areas/ questions that we would like to provide insights to the stakeholders, we will further broaden/alter the scope of analysis as we go through the data mining exercise. As an outcome of this exercise, we would like to create a framework for the commuters and taxi driver/owners that helps them to plan their trips across various time points, locations and weather conditions

# Datasets and Data Sources

## <b>Data Source 1</b> : ##

> NYC Open Data -  New York Yellow & Green Taxi Trip data (Timestamp level)
  1. Size: N/A (As this is API based, we will only get details once the entire data is pulled)
  2. Format: API (JSON)
  3. Access method: Python API calls & Google BigQuery framework

> 
Dataset Name | Link
--- | ---
Yellow Taxi (2019) | https://data.cityofnewyork.us/Transportation/2019-Yellow-Taxi-Trip-Data/2upf-qytp 
Yellow Taxi (2018) | https://data.cityofnewyork.us/Transportation/2018-Yellow-Taxi-Trip-Data/t29m-gskq
Yellow Taxi (2017) | https://data.cityofnewyork.us/Transportation/2017-Yellow-Taxi-Trip-Data/biws-g3hs 
Green Taxi (2019) | https://data.cityofnewyork.us/Transportation/2019-Green-Taxi-Trip-Data/q5mz-t52e 
Green Taxi (2018) | https://data.cityofnewyork.us/Transportation/2018-Green-Taxi-Trip-Data/w7fs-fd9i 
Green Taxi (2017) | https://data.cityofnewyork.us/Transportation/2017-Green-Taxi-Trip-Data/5gj9-2kzx 

## <b>Data Source 2</b> : ##

> NYC Open Data -  National Centers for Environmental Information
  1. Size: 17MB
  2. Format: CSV
  3. Access method: HTTP
  4. Location : https://www.ncdc.noaa.gov




## <b> Data Source 3 : </b>##

> NYC Open Data -  Taxi Zone information
  1. Size: 1MB
  2. Format: Shapefile
  3. Access method: HTTP
  4. Location : https://www1.nyc.gov/site/tlc/about/tlc-trip-record-data.page
  

# Data Exploration

We will first start with data exploration - Starting with Taxi zone information

Let's first mount the working location. In our case, we used the Google Drive infrastructure to complete this project and required dependencies

In [2]:
from google.colab import drive
from os.path import join

ROOT = '/content/drive'
PROJ = 'MyDrive/Milestones/Milestone1'

drive.mount(ROOT)
PROJECT_PATH = join(ROOT, PROJ)
%cd "$PROJECT_PATH"
%pwd

Mounted at /content/drive
/content/drive/MyDrive/Milestones/Milestone1


'/content/drive/MyDrive/Milestones/Milestone1'

In [3]:
!pip install geopandas
!pip install altair_data_server

import io
import json
import pandas as pd
import numpy as np
import geopandas as gpd
from shapely.ops import cascaded_union
from google.cloud import bigquery
from google.oauth2 import service_account
import datetime as dt
import altair as alt

alt.data_transformers.enable('data_server')

     |████████████████████████████████| 972kB 9.6MB/s 
     |████████████████████████████████| 14.8MB 325kB/s 
     |████████████████████████████████| 6.5MB 22.3MB/s 


DataTransformerRegistry.enable('data_server')

Understanding the <b> Taxi Zones </b> dataset

In [4]:
taxi_zones = gpd.read_file('https://s3.amazonaws.com/nyc-tlc/misc/taxi_zones.zip')
taxi_zones.to_crs(epsg=4326, inplace=True)
taxi_zones['centroid_lon'] = taxi_zones['geometry'].centroid.x
taxi_zones['centroid_lat'] = taxi_zones['geometry'].centroid.y

taxi_zones_b = taxi_zones.groupby('borough')['geometry'].agg(lambda x: cascaded_union(x).centroid).to_frame()
taxi_zones_b.columns = ['geometry']
taxi_zones_b.reset_index(inplace=True)
taxi_zones_b['centroid_lon'] = taxi_zones_b['geometry'].centroid.x
taxi_zones_b['centroid_lat'] = taxi_zones_b['geometry'].centroid.y


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:3: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  This is separate from the ipykernel package so we can avoid doing imports until
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:4: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  after removing the cwd from sys.path.


In [5]:
tz_geo = json.loads(taxi_zones.to_json())['features']
tz_geo_b = json.loads(taxi_zones_b.to_json())['features']

alt.themes.enable('opaque')

base = alt.Chart(alt.Data(values=tz_geo)).mark_geoshape(
        stroke='black',
        strokeWidth=1
    ).encode(
        color=alt.Color('properties.borough:N', legend=None)
    ).properties(
        width=800,
        height=800
    )

labels = alt.Chart(alt.Data(values=tz_geo)).mark_text(
    baseline='top',
     ).properties(
        width=800,
        height=800
     ).encode(
         longitude='properties.centroid_lon:Q',
         latitude='properties.centroid_lat:Q',
         text='properties.LocationID:O',
         size=alt.value(8),
         opacity=alt.value(1)
     )

boroughs = alt.Chart(alt.Data(values=tz_geo_b)).mark_text(
    color='white',
    stroke='black',
    fontWeight='bold',
    strokeWidth=0.7,
    baseline='top'
     ).properties(
        width=800,
        height=800,
        title=alt.Text(text="NYC boroughs and taxi zones", fontSize=22)
     ).encode(
         longitude='properties.centroid_lon:Q',
         latitude='properties.centroid_lat:Q',
         text='properties.borough:N',
         size=alt.value(26),
         opacity=alt.value(1)
     )


base + labels + boroughs

Output hidden; open in https://colab.research.google.com to view.

In [6]:
key_path = 'auth.json'
credentials = service_account.Credentials.from_service_account_file(key_path)
client = bigquery.Client(credentials=credentials, project=credentials.project_id)

sql = '''SELECT dropoff_location_id, count(*) as count
FROM bigquery-public-data.new_york_taxi_trips.tlc_yellow_trips_2017 
where dropoff_datetime between '2017-01-01' and '2018-01-01' 
group by dropoff_location_id;'''
dropoff_2017_df = client.query(sql).to_dataframe()

In [7]:
dropoff_2017_df.rename(columns={'dropoff_location_id': 'LocationID'}, inplace=True)
dropoff_2017_df['LocationID'] = dropoff_2017_df['LocationID'].astype('int64')

dropoff_2017 = taxi_zones.merge(dropoff_2017_df, on='LocationID')
dropoff_2017 = json.loads(dropoff_2017.to_json())['features']

base = alt.Chart(alt.Data(values=dropoff_2017)).mark_geoshape(
        stroke='black',
        strokeWidth=1
    ).encode(
        color=alt.Color('properties.count:Q', scale=alt.Scale(type='log'), legend=alt.Legend(title="Drop-off count"))
    ).properties(
        title=alt.Text(text="NYC taxi drop-off zones popularity", fontSize=22),
        width=800,
        height=800
    )

labels = alt.Chart(alt.Data(values=tz_geo)).mark_text(
    baseline='top',
     ).properties(
        width=800,
        height=800
     ).encode(
         longitude='properties.centroid_lon:Q',
         latitude='properties.centroid_lat:Q',
         text='properties.LocationID:O',
         size=alt.value(8),
         opacity=alt.value(1)
     )

base + labels

Output hidden; open in https://colab.research.google.com to view.

In [9]:
key_path = 'auth.json'
credentials = service_account.Credentials.from_service_account_file(key_path)

sql = '''SELECT pickup_location_id, count(*) as count
FROM bigquery-public-data.new_york_taxi_trips.tlc_yellow_trips_2017 
where pickup_datetime between '2017-01-01' and '2018-01-01' 
group by pickup_location_id;'''
pickup_2017_df = client.query(sql).to_dataframe()

In [10]:
pickup_2017_df.rename(columns={'pickup_location_id': 'LocationID'}, inplace=True)
pickup_2017_df['LocationID'] = pickup_2017_df['LocationID'].astype('int64')

pickup_2017 = taxi_zones.merge(pickup_2017_df, on='LocationID')
pickup_2017 = json.loads(pickup_2017.to_json())['features']

base = alt.Chart(alt.Data(values=pickup_2017)).mark_geoshape(
        stroke='black',
        strokeWidth=1
    ).encode(
        color=alt.Color('properties.count:Q', scale=alt.Scale(type='log'), legend=alt.Legend(title="Pickup count"))
    ).properties(
        title=alt.Text(text="NYC taxi pickup zones popularity", fontSize=22),
        width=800,
        height=800
    )
    
labels = alt.Chart(alt.Data(values=tz_geo)).mark_text(
    baseline='top',
     ).properties(
        width=800,
        height=800
     ).encode(
         longitude='properties.centroid_lon:Q',
         latitude='properties.centroid_lat:Q',
         text='properties.LocationID:O',
         size=alt.value(8),
         opacity=alt.value(1)
     )

base + labels

Output hidden; open in https://colab.research.google.com to view.

In [32]:
sql = '''
SELECT 
dropoff_location_id, avg(fare_amount) as avg_fare
FROM 
bigquery-public-data.new_york_taxi_trips.tlc_yellow_trips_2018
WHERE dropoff_datetime > '2018-01-01' and dropoff_datetime < '2019-01-01'
AND fare_amount > 0 and fare_amount < 1000
GROUP BY dropoff_location_id;
'''
avg_fare_2018_df = client.query(sql).to_dataframe()

In [12]:
avg_fare_2018_df.rename(columns={'dropoff_location_id': 'LocationID'}, inplace=True)
avg_fare_2018_df['LocationID'] = avg_fare_2018_df['LocationID'].astype('int64')
avg_fare_2018_df['avg_fare'] = avg_fare_2018_df['avg_fare'].astype('float64')

avg_fare_2018 = taxi_zones.merge(avg_fare_2018_df, on='LocationID')
avg_fare_2018 = json.loads(avg_fare_2018.to_json())['features']

base = alt.Chart(alt.Data(values=avg_fare_2018)).mark_geoshape(
        stroke='black',
        strokeWidth=1
    ).encode(
        color=alt.Color('properties.avg_fare:Q', legend=alt.Legend(title="Avg. fare"))
    ).properties(
        title=alt.Text(text="NYC average fare by taxi zone", fontSize=22),
        width=800,
        height=800
    )
    
labels = alt.Chart(alt.Data(values=tz_geo)).mark_text(
    baseline='top',
     ).properties(
        width=800,
        height=800
     ).encode(
         longitude='properties.centroid_lon:Q',
         latitude='properties.centroid_lat:Q',
         text='properties.LocationID:O',
         size=alt.value(8),
         opacity=alt.value(1)
     )

base + labels

Output hidden; open in https://colab.research.google.com to view.

In [34]:
temperature = pd.read_csv('/content/drive/My Drive/Milestones/Temperature Data.csv', low_memory=False)